Librerías y conexión a la base de datos

Se importan las librerías para las bases de datos

In [225]:
import pandas as pd
import numpy as np
#import sqlite3
from openpyxl import Workbook
#from dbi import connect
import pyodbc
#from pyodbc import connect as pyodbc_connect
#from sqlite3 import connect as sqlite3_connect
from scipy import stats
from scipy import special
from scipy.stats import gmean

In [226]:
import funciones as f

In [227]:
anio = int(input('Ingrese el año: '))
mes = int(input('Ingrese el mes: '))

#anio = 2024
#mes = 3

In [228]:
#Almacenamos la información del servidor para poder realizar la conexion más adelante
server = 'ipcprod.database.windows.net'
user = 'ipcreader'
password = '1pc/*2023'
database = 'db-indices'

In [229]:
#Se crea un string para realizar la conexión, con la informacion del servidor almacenada previamente
conn_str = (
    f'Driver={{ODBC Driver 17 for SQL Server}};'
    f'Server={server};'
    f'Database={database};'
    f'Uid={user};'
    f'Pwd={password};'
)

In [230]:
try:
    conn = pyodbc.connect(conn_str)
    cursor = conn.cursor()
    
    sql_command = f"EXEC [dbo].[sp_get_precios_recolectados_mes] {anio}, {mes}"
    cursor.execute(sql_command)
    boletas = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])
    
    sql_command01 = f"EXEC sp_get_indice_grupo {anio}, {mes}"
    cursor.execute(sql_command01)
    ponderaciones = pd.DataFrame([tuple(row) for row in cursor.fetchall()], columns=[column[0] for column in cursor.description])

    conn.close()
    print("Connection successful!")
except Exception as e:
    print(f"Error: {e}")

Connection successful!


In [231]:
#boletas

___________________________________________________________________________________________

In [232]:
participacion = pd.DataFrame({
        'region': [1, 2, 3, 4, 5, 6, 7, 8],
        'participacion': [13.472203, 12.046949, 12.486967, 12.483411, 12.674423, 12.415842, 12.190962, 12.229243]
    })
participacion['region'] = participacion['region'].astype('int64')

In [233]:
bol = boletas[['region', 'codigo_articulo', 'articulo', 'cantidad_anterior', 'cantidad_actual', 'precio_anterior', 'precio_actual', 'nt_tipo']].copy()
bol.insert(1, 'cod_prod', np.where(bol['codigo_articulo'].str.len() > 8, 
                                   bol['codigo_articulo'].str.slice(0, -2), 
                                   bol['codigo_articulo'].str.slice(0, -1)))
bol.loc[:, 'cod_prod'] = bol['cod_prod'].apply(lambda x: '0' + x if len(x) == 6 else x)
bol['cantidad_anterior'] = pd.to_numeric(bol['cantidad_anterior'], errors='coerce')
bol['cantidad_actual'] = pd.to_numeric(bol['cantidad_actual'], errors='coerce')
bol['precio_anterior'] = pd.to_numeric(bol['precio_anterior'], errors='coerce')
bol['precio_actual'] = pd.to_numeric(bol['precio_actual'], errors='coerce')
bol.loc[bol['cantidad_anterior'] == 0 | bol['cantidad_anterior'].isnull(), 'cantidad_anterior'] = bol['cantidad_actual']
bol.loc[bol['precio_anterior'] == 0 | bol['precio_anterior'].isnull(), 'precio_anterior'] = bol['precio_actual']

#bol['cantidad_actual'] = bol['cantidad_anterior']
#bol['precio_actual'] = bol['precio_anterior']

bol

,region,cod_prod,codigo_articulo,articulo,cantidad_anterior,cantidad_actual,precio_anterior,precio_actual,nt_tipo
0,1,0540301,5403012,Olla y cacerola de 1 a 2lt,1.0,1.0,145.00,145.00,NaN
1,1,0312306,3123061,Vestidos formales con mangas de largo hasta la...,1.0,1.0,250.00,250.00,NaN
2,1,0721101,7211011,Llantas para automóvil 185/65 R14,1.0,1.0,375.00,375.00,NaN
3,1,0411101,4111013,Alquiler de apartamento (con o sin servicios),1.0,1.0,1700.00,1700.00,NaN
4,1,0321105,3211051,Sandalias para hombre,1.0,1.0,279.99,279.99,NaN
...,...,...,...,...,...,...,...,...,...
71315,8,0454202,4542021,Manojo de ocote,1.0,1.0,5.00,5.00,NaN
71316,8,0732202,7322021,"Alquiler de vehículo por día, kilometraje ilim...",1.0,1.0,406.00,406.00,NaN
71317,8,0961001,9610011,"Boleto de cine 2D, sala normal, precio fin de ...",1.0,1.0,35.00,35.00,NaN
71318,8,0961001,9610012,"Boleto de cine 3D, sala normal, precio fin de ...",1.0,1.0,45.00,45.00,NaN


In [234]:
# Se obtienen las ponderaciones de cada produto por cada region y los indices anteriores
pon_prod = f.pondcat('Producto', ponderaciones).rename(columns={'grupo_codigo':'cod_prod'})
pon_prod['cod_prod'] = pon_prod['cod_prod'].apply(lambda x: '0' + x if len(str(x)) == 6 else x)
pon_prod['ponderacion_region'] = pd.to_numeric(pon_prod['ponderacion_region'], errors='coerce')
pon_prod['indice_anterior'] = pd.to_numeric(pon_prod['indice_anterior'], errors='coerce')
#pon_prod

In [235]:
#pon_prod['ponderacion_region'].dtype
#pon_prod['ponderacion_region'].isnull().sum()

Calculo del IPC

In [236]:
# Se calcula la variacion de las variedades
variacion = f.variacion(bol)
variacion['var_prod'] = bol.apply(lambda row: 1 if row['nt_tipo'] in [351761, 351762] else row['var_prod'], axis=1)
variacion = variacion[['region', 'cod_prod', 'codigo_articulo', 'articulo', 'var_prod']]
#variacion['var_prod'].dtype
#variacion

In [237]:
variacion.to_excel('variacionpy.xlsx')

In [238]:
#relativo de precios de los productos
relativo = (variacion.groupby(['region', 'cod_prod'])
            .agg(relativo = ('var_prod', lambda x: gmean(x)))
            .reset_index())
relativo = pd.merge(relativo, pon_prod[['region', 'cod_prod', 'grupo_nombre', 'ponderacion_region', 'indice_anterior']],
         on=['region', 'cod_prod'], how='left')
relativo
#relativo['ponderacion_region'].dtype

#relativo[relativo['ponderacion_region'] == 0]

,region,cod_prod,relativo,grupo_nombre,ponderacion_region,indice_anterior
0,1,0111101,1.009423,Arroz,0.399291,104.394816
1,1,0111102,0.968050,Maíz,0.130256,99.221683
2,1,0111201,0.999530,Harina de trigo,0.072427,97.976553
3,1,0111202,0.984028,Harina de maíz,0.039212,104.682837
4,1,0111203,0.986929,Harina para atoles,0.066483,103.539063
...,...,...,...,...,...,...
3142,8,1329101,1.000000,Sombrillas y paraguas,0.050085,100.000000
3143,8,1329102,1.000000,Ataúdes,0.065264,100.000000
3144,8,1390901,1.000000,Servicios legales y financieros,0.194022,100.000000
3145,8,1390902,1.000000,Servicios funerarios,0.037005,100.000000


In [239]:
#nulos = relativo['ponderacion_region'].isnull()
#relativo[nulos]
relativo.to_excel('relativopy.xlsx')

In [240]:
#promedio ponderado
prom_pon = relativo
prom_pon['var_ac'] = prom_pon['relativo']*prom_pon['ponderacion_region']
prom_pon = prom_pon[prom_pon['var_ac'] != 0]
## lo unico que se me ocurre es hacer otros dataframes agrupados, uno para la suma de la var ac, otro para la suma de las ponderaciones y asi y luego unirlos

#prom_pon['sum_var_ac'] = prom_pon['var_ac'].sum()
prom_pon01 = prom_pon.groupby(['region', 'cod_prod', 'grupo_nombre']).agg(
    sum_var_ac=('var_ac', 'sum'),
    sum_ponderacion=('ponderacion_region', 'sum')
    ).reset_index()
prom_pon = prom_pon.merge(prom_pon01, on=['region', 'cod_prod', 'grupo_nombre'], how='left')
prom_pon['prom_pon'] = prom_pon['sum_var_ac']/prom_pon['sum_ponderacion']

#prom_pon
#prom_pon01
#prom_pon[prom_pon['var_ac'] == 0]
#prom_pon01[prom_pon01['sum_var_ac'] == 0]

In [241]:
prom_pon.to_excel('promponpy.xlsx')

Índice de precios por región

In [242]:
#índice productos
prom_pon['ind_prod'] = prom_pon['indice_anterior']*prom_pon['prom_pon']
ind_prod = f.cod_grupos(prom_pon).dropna()

ind_prod01 = ind_prod[['region', 'cod_prod', 'ind_prod']].copy()
#dist100['ind_prod'].dtype

In [243]:
ind_prod01.to_excel('indprodpy.xlsx')

In [244]:
#indice subclase
f.indices(ind_prod,'cod_subclase')

,region,grupo_codigo,ind
0,1,01111,103.084284
1,1,01112,100.637127
2,1,01113,99.399761
3,1,01114,100.520526
4,1,01115,98.258502
...,...,...,...
1222,8,13131,102.433519
1223,8,13132,100.000000
1224,8,13211,100.000000
1225,8,13291,100.000000


In [245]:
#indice clase
f.indices(ind_prod,'cod_clase')

,region,grupo_codigo,ind
0,1,0111,99.714823
1,1,0112,99.403764
2,1,0113,101.053302
3,1,0114,100.329696
4,1,0115,101.952381
...,...,...,...
685,8,1312,101.481221
686,8,1313,101.969290
687,8,1321,100.000000
688,8,1329,100.000000


In [246]:
#indice grupo
f.indices(ind_prod,'cod_grupo')

,region,grupo_codigo,ind
0,1,011,100.592131
1,1,012,100.435152
2,1,013,100.000000
3,1,021,98.033709
4,1,023,102.251526
...,...,...,...
327,8,111,100.232556
328,8,112,105.057279
329,8,131,101.748291
330,8,132,100.000000


In [247]:
#indice division
f.indices(ind_prod,'cod_div')

,region,grupo_codigo,ind
0,1,01,100.578409
1,1,02,99.191826
2,1,03,100.545126
3,1,04,99.910533
4,1,05,99.923276
...,...,...,...
95,8,08,100.059956
96,8,09,99.811660
97,8,10,100.291196
98,8,11,100.261477


Índice a nivel republica

In [248]:
#indice productos
f.indicerep(ind_prod,'cod_prod')

,cod_prod,ind_prod_rep
0,0111101,102.850585
1,0111102,98.651088
2,0111201,97.550605
3,0111202,101.173660
4,0111203,100.816421
...,...,...
432,1390901,100.000000
433,1390902,101.421375
434,1390903,100.000000
435,1390904,100.000000


In [249]:
#indice subclase
f.indicerep(ind_prod,'cod_subclase')

,cod_subclase,ind_prod_rep
0,01111,100.328624
1,01112,100.559483
2,01113,99.119302
3,01114,100.534328
4,01115,99.622538
...,...,...
163,13131,101.340305
164,13132,100.289312
165,13211,100.277803
166,13291,100.000000


In [250]:
#indice clase
f.indicerep(ind_prod,'cod_clase')

,cod_clase,ind_prod_rep
0,0111,99.423776
1,0112,100.612409
2,0113,100.538732
3,0114,97.535490
4,0115,102.107144
...,...,...
91,1312,99.669568
92,1313,101.227079
93,1321,100.277803
94,1329,100.000000


In [251]:
#indice grupo
f.indicerep(ind_prod,'cod_grupo')

,cod_grupo,ind_prod_rep
0,011,100.743654
1,012,100.917847
2,013,100.673214
3,021,99.232963
4,023,100.196202
5,031,100.380101
6,032,100.722321
7,041,100.386798
8,043,100.106672
9,044,103.874719


In [252]:
#indice division
f.indicerep(ind_prod,'cod_div')

,cod_div,ind_prod_rep
0,01,100.757739
1,02,99.428571
2,03,100.468190
3,04,99.540459
4,05,100.186334
5,06,100.440593
6,07,104.660353
7,08,100.227833
8,09,100.320680
9,10,100.418739


Indice general

In [253]:
#indice producos a nivel nacional
def indicenac(df):
    indice = df.groupby('region').apply(lambda x: pd.Series({
        'sum1': (x['ponderacion_region'] * x['ind_prod']).sum(),
        'sum2': x['ponderacion_region'].sum()
    })).reset_index()

    indice['ind_prod_rep'] = indice['sum1'] / indice['sum2']
    indice = indice[['region', 'ind_prod_rep']]

    return indice



In [254]:
indicenac(ind_prod)

,region,ind_prod_rep
0,1,100.906771
1,2,100.637088
2,3,100.905612
3,4,101.118868
4,5,100.489914
5,6,100.716112
6,7,100.714111
7,8,101.081104


In [255]:
def calculate_replicated_indicator(df):
    sum1 = (df['ponderacion_region'] * df['ind_prod']).sum()
    sum2 = df['ponderacion_region'].sum()

    result = pd.DataFrame({
        'sum1': [sum1],
        'sum2': [sum2]
    })

    result['ind_rep'] = result['sum1'] / result['sum2']

    return result

In [256]:
calculate_replicated_indicator(ind_prod)

,sum1,sum2,ind_rep
0,73988.195679,733.898247,100.815332
